#### 기존 openai-251127-05-함수호출-통합과제.ipynb에서는 도우미(assistant) 사용하지 않았음. 
#### 챗봇과 사용자간의 스레드 생성 시, 콜백 함수로 get_current_weather을 지정하였으므로, 사용자 질의 시 첫번 째 상태가 requires_action로 응답됨
#### requires_action 상태에서, 콜백 함수 호출 후, 결과를 바탕으로 다시 제출하여 

In [ ]:
import os
import json
import requests
import time
from openai import AzureOpenAI

from dotenv import load_dotenv
# .env 환경변수 로드
load_dotenv()


 
# OpenWeatherMap API 키 (날씨 정보 조회에 사용)
OPEN_WEATHER_API_KEY = os.getenv("OPEN_WEATHER_API_KEY")


def get_current_weather(location: str, unit: str = "celsius") -> str:
    """
    주어진 도시의 현재 날씨 정보를 조회합니다.
    location은 도시 이름이며, unit은 온도 단위입니다 (기본값: celsius).
    """
    print(f"\n--- [Tool Call: get_current_weather] ---")
    print(f"Location: {location}, Unit: {unit}")
    
    # API 키가 환경 변수에 설정되어 있는지 확인
    if not OPEN_WEATHER_API_KEY:
        error_message = "OpenWeatherMap API Key가 설정되지 않았습니다."
        print(error_message)
        return json.dumps({"error": error_message})

    # 1단계: 도시 이름으로 위도 및 경도(Latitude, Longitude) 조회 (Geocoding API)
    # OpenWeatherMap API 키 변수 사용
    geo_endpoint = f"http://api.openweathermap.org/geo/1.0/direct?q={location}&appid={OPEN_WEATHER_API_KEY}"
    try:
        geo_response = requests.get(geo_endpoint, timeout=5)
        geo_response.raise_for_status() # HTTP 오류 발생 시 예외 발생
        
        geo_json = geo_response.json()
        if not geo_json:
            return json.dumps({"location": location, "weather": "City not found."})
        
        # 첫 번째 검색 결과 사용
        lat = geo_json[0].get('lat')
        lon = geo_json[0].get('lon')
        
        if lat is None or lon is None:
            return json.dumps({"location": location, "weather": "Invalid coordinates received."})

        # 2단계: 위경도를 사용하여 현재 날씨 정보 조회
        units_param = "metric" if unit.lower() == "celsius" else "imperial"
        # OpenWeatherMap API 키 변수 사용
        weather_endpoint = (
            f"https://api.openweathermap.org/data/2.5/weather?"
            f"units={units_param}&lat={lat}&lon={lon}&appid={OPEN_WEATHER_API_KEY}"
        )

        weather_response = requests.get(weather_endpoint, timeout=5)
        weather_response.raise_for_status()
        
        weather_data = weather_response.json()
        
        # 필요한 정보 추출
        temperature = weather_data.get('main', {}).get('temp')
        weather_desc = weather_data.get('weather', [{}])[0].get('description')
        
        result = {
            "location": weather_data.get('name', location),
            "temperature": temperature,
            "unit": unit.lower(),
            "description": weather_desc
        }
        return json.dumps(result)

    except requests.exceptions.Timeout:
        return json.dumps({"location": location, "error": "API request timed out."})
    except requests.exceptions.RequestException as e:
        print(f"Weather API request failed: {e}")
        return json.dumps({"location": location, "error": f"API request failed: {e}"})
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return json.dumps({"location": location, "error": f"An unexpected error occurred: {e}"})

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OAI_ENDPOINT"),
  api_key= os.getenv("AZURE_OAI_KEY"),
  api_version="2024-05-01-preview"
)

# 콜백 함수 get_current_weather로 지정하여, 1차 Open AI 호출 시 상태가 requires_action으로 출력됨
# requires_action 상태일 때, 콜백 함수로 지정한 get_current_weather호출하여 결과(날씨) 받아온 후, 다시 제출하는 코드 작성 필요해보임
assistant = client.beta.assistants.create(
  model="gpt-4o-mini", # replace with model deployment name.
  instructions="당신은 날씨를 알려주는 봇입니다. 모르는 지역은 솔직히 모른다고 하세요.",
  tools=[{"type":"function","function":{"name":"get_current_weather","description":"지역의 날씨를 조회합니다.","parameters":{"type":"object","properties":{"location":{"type":"string"}},"required":["location"]},"strict":False}}],
  tool_resources={},
  temperature=1,
  top_p=1
)
 
# 사용자와 도우미 간의 대화 세션인 스레드(Thread)를 새로 생성
thread = client.beta.threads.create()
 
# 생성된 스레드(thread.id)에 사용자(role="user")의 질문을 메시지로 추가
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="서울 날씨 알려줘"
)
 
  
# 도우미(assistant.id)에게 스레드(thread.id)의 메시지를 처리하도록 요청하는 실행을 시작
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)


# completed, failed, expired 상태가 될 때까지 반복
# queued : 대기중, in_progress : 실행중
while run.status not in ['completed', 'failed', 'expired']:
    
    # 1. requires_action 상태 처리 (함수 실행 요청 시)
    if run.status == 'requires_action':
        tool_outputs = []
        
        # Run이 요구하는 함수 호출 목록 순회
        for tool_call in run.required_action.submit_tool_outputs.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            tool_call_id = tool_call.id

            if function_name == "get_current_weather":
                # 콜백 함수 실행
                result = get_current_weather(
                    location=function_args.get("location"),
                    unit=function_args.get("unit", "celsius")
                )
                
                print(f"get_current_weather Result :  {function_args.get('location')}")
                
                # get_current_weather  결과를 tool_outputs 리스트에 추가
                tool_outputs.append({
                    "tool_call_id": tool_call_id,
                    "output": result
                })
        
        # 함수 실행 결과를 Run에 제출하고, Run을 다시 in_progress 상태로 전환
        run = client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_outputs
        )

    # 2. 다른 상태 (queued, in_progress, cancelling)일 경우 대기 및 상태 업데이트
    elif run.status in ['queued', 'in_progress', 'cancelling']:
        
        print(f"Status: {run.status}. Waiting 1 second...")
        time.sleep(1)

        # Run 상태 다시(retrieve) 조회. 즉, 다시 제출
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )
    
    # 3. 예외적인 상태가 발생했으나 위에서 처리되지 않은 경우 (거의 발생하지 않음)
    else:
        print(f"Unexpected status encountered: {run.status}")
        break


# --- 최종 결과 처리 ---
if run.status == 'completed':
    print("\n Run Completed!")
    messages = client.beta.threads.messages.list(
        thread_id=thread.id,
        order="desc" # 최신 메시지(답변)를 먼저 가져옴
    )
    # messages.data[0]는 도우미의 최종 답변입니다.
    assistant_response = messages.data[0].content[0].text.value
    print("-" * 50)
    print(f"Assistant Response:\n{assistant_response}")
    print("-" * 50)
    
else:
    print(f"\n Run Failed or Expired. Final Status: {run.status}")




C:\Users\EL32\AppData\Local\Temp\ipykernel_17120\1471595978.py:100: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
C:\Users\EL32\AppData\Local\Temp\ipykernel_17120\1471595978.py:103: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.create(
C:\Users\EL32\AppData\Local\Temp\ipykernel_17120\1471595978.py:111: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(


Status: queued. Waiting 1 second...


C:\Users\EL32\AppData\Local\Temp\ipykernel_17120\1471595978.py:159: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(


Status: in_progress. Waiting 1 second...

--- [Tool Call: get_current_weather] ---
Location: 서울, Unit: celsius

--- [Function Output Submitted] ---
Result for 서울: {"location": "Seoul", "temperature": 6.76, "unit":...


C:\Users\EL32\AppData\Local\Temp\ipykernel_17120\1471595978.py:146: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.submit_tool_outputs(


Status: queued. Waiting 1 second...
Status: in_progress. Waiting 1 second...

✅ Run Completed!


C:\Users\EL32\AppData\Local\Temp\ipykernel_17120\1471595978.py:173: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages = client.beta.threads.messages.list(


--------------------------------------------------
Assistant Response:
서울의 현재 날씨는 맑은 하늘이며, 기온은 6.76도입니다.
--------------------------------------------------
